<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/FinalResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import math

# Preprocessing the data

In [21]:
!wget https://github.com/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/data/FullAndFracture.zip

--2022-11-25 23:06:51--  https://github.com/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/data/FullAndFracture.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘FullAndFracture.zip.1’

FullAndFracture.zip     [ <=>                ] 143.71K   781KB/s    in 0.2s    

2022-11-25 23:06:51 (781 KB/s) - ‘FullAndFracture.zip.1’ saved [147158]



In [22]:
!unzip FullAndFracture.zip

Archive:  FullAndFracture.zip
  inflating: FullAndFracture.npy     


In [23]:
data = np.load('FullAndFracture.npy', allow_pickle=True)

In [24]:
data = np.array(data, dtype = np.int32)

In [25]:
X = data[1]
Y = data[0]

In [26]:
lx = X.shape[0]
ly = Y.shape[0]

In [27]:
X = list(X)
Y = list(Y)

In [28]:
#flattening each chair
for i in range(lx):
  X[i] = X[i].flatten()
for i in range(ly):
  Y[i] = Y[i].flatten()

In [29]:
X, Y = np.array(X), np.array(Y)

In [35]:
train = 7112 * 0.7 // 1
train = int(train)

In [71]:
x_train, x_val, y_train, y_val = X[:train], X[train:], Y[:train], Y[train:]

In [38]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(4978, 32768) (4978, 32768)
(2134, 32768) (2134, 32768)


# Implementing, traing and saving the model

In [39]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(32768,)))
model.add(tf.keras.layers.Dense(768, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(768, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32768))

model.compile(optimizer = 'adam',
         loss = 'mean_squared_error',
         metrics = ['mse'])

model.fit(x_train, y_train, epochs = 200)

Epoch 1/200
156/156 [==============================] - 84s 533ms/step - loss: 0.0357 - mse: 0.0357
Epoch 2/200
156/156 [==============================] - 81s 522ms/step - loss: 0.0223 - mse: 0.0223
Epoch 3/200
156/156 [==============================] - 83s 532ms/step - loss: 0.0160 - mse: 0.0160
Epoch 4/200
156/156 [==============================] - 81s 521ms/step - loss: 0.0122 - mse: 0.0122
Epoch 5/200
156/156 [==============================] - 83s 535ms/step - loss: 0.0101 - mse: 0.0101
Epoch 6/200
156/156 [==============================] - 83s 530ms/step - loss: 0.0085 - mse: 0.0085
Epoch 7/200
156/156 [==============================] - 82s 526ms/step - loss: 0.0078 - mse: 0.0078
Epoch 8/200
156/156 [==============================] - 83s 531ms/step - loss: 0.0074 - mse: 0.0074
Epoch 9/200
156/156 [==============================] - 82s 524ms/step - loss: 0.0071 - mse: 0.0071
Epoch 10/200
156/156 [==============================] - 83s 530ms/step - loss: 0.0069 - mse: 0.0069
Epoch 11/

In [52]:
model.save('FinalModel')

In [54]:
!zip -r /content/FinalModel.zip /content/FinalModel

  adding: content/FinalModel/ (stored 0%)
  adding: content/FinalModel/keras_metadata.pb (deflated 87%)
  adding: content/FinalModel/saved_model.pb (deflated 88%)
  adding: content/FinalModel/assets/ (stored 0%)
  adding: content/FinalModel/variables/ (stored 0%)
  adding: content/FinalModel/variables/variables.index (deflated 63%)
  adding: content/FinalModel/variables/variables.data-00000-of-00001 (deflated 24%)


# Getting the predictions

In [64]:
prediction = model.predict(x_val)

67/67 [==============================] - 6s 92ms/step


In [68]:
prediction = prediction.round()
prediction = np.array(prediction, dtype=np.int32)

# Ploting Functions

In [129]:
def plot3d(verts, s=10, c=(105,127,155), show_grid=False):
    x, y, z = zip(*verts)
    color = f'rgb({c[0]}, {c[1]}, {c[2]})'
    trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=s,
            color=color,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    data = [trace]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene = go.Scene(
            xaxis=dict(visible=show_grid),
            yaxis=dict(visible=show_grid),
            zaxis=dict(visible=show_grid)
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
def point_cloud_to_volume(points, vsize, radius=1.0):
    vol = np.zeros((vsize,vsize,vsize), dtype=np.bool)
    voxel = 2*radius/float(vsize)
    locations = (points + radius)/voxel
    locations = locations.astype(int)
    vol[locations[:,0],locations[:,1],locations[:,2]] = 1.0
    
    return vol
    
def volume_to_point_cloud(vol):
    """ vol is occupancy grid (value = 0 or 1) of size vsize*vsize*vsize
        return Nx3 numpy array.
    """
    vsize = vol.shape[0]
    assert(vol.shape[1] == vsize and vol.shape[1] == vsize)
    points = []
    for a in range(vsize):
        for b in range(vsize):
            for c in range(vsize):
                if vol[a,b,c] == 1:
                    points.append(np.array([a,b,c]))
    if len(points) == 0:
        return np.zeros((0,3))
    points = np.vstack(points)
    
    return points

def auto_pcl_to_volume(points, vsize):
	data_min = np.min(points)
	data_max = np.max(points)
	radius = max(abs(data_min), data_max)
	radius = math.ceil(radius*100) / 100
	vol = point_cloud_to_volume(points, vsize, radius)

	return vol

def plot_vol(vol, s=10, c=(105,127,155), show_grid=False):
    if vol.dtype != np.bool:
        vol = vol > 0

    pc = volume_to_point_cloud(vol)
    plot3d(pc, s, c, show_grid)

def plot_reconstruction(vol1, vol2, s=10,
                        c2=(105,128,155), c1=(182,49,62), show_grid=False):
    if vol1.dtype != np.bool:
        vol1 = vol1 > 0
    if vol2.dtype != np.bool:
        vol2 = vol2 > 0
        
    color1 = f'rgb({c1[0]}, {c1[1]}, {c1[2]})'
    color2 = f'rgb({c2[0]}, {c2[1]}, {c2[2]})'
    vol2 = np.logical_xor(vol2, vol1)
    pc1 = volume_to_point_cloud(vol1)
    pc2 = volume_to_point_cloud(vol2)
    x1, y1, z1 = zip(*pc1)
    x2, y2, z2 = zip(*pc2)
    trace1 = go.Scatter3d(
        x=x1, y=y1, z=z1,
        mode='markers',
        marker=dict(
            size=s,
            color=color1,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    trace2 = go.Scatter3d(
        x=x2, y=y2, z=z2,
        mode='markers',
        marker=dict(
            size=s,
            color=color2,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    data = [trace1, trace2]
    layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                       scene = go.Scene(
                               xaxis=dict(visible=show_grid),
                               yaxis=dict(visible=show_grid),
                               zaxis=dict(visible=show_grid)),
                       showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

# Ploting the results

In [121]:
from random import randint
k = randint(0, 2133)
plot_reconstruction((prediction[k] - x_val[k]).reshape(32,32,32), prediction[k].reshape(32,32,32), s = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



# Prepare the results to be printed

In [108]:
!pip install voxelfuse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63.0 MB 8.6 kB/s 
     |████████████████████████████████| 129 kB 61.1 MB/s 
     |████████████████████████████████| 995 kB 48.2 MB/s 
     |████████████████████████████████| 8.4 MB 39.9 MB/s 
     |████████████████████████████████| 15.1 MB 34.1 MB/s 
     |████████████████████████████████| 167 kB 51.3 MB/s 
     |████████████████████████████████| 265 kB 52.5 MB/s 
     |████████████████████████████████| 237 kB 47.2 MB/s 
     |████████████████████████████████| 59.9 MB 1.4 MB/s 
     |████████████████████████████████| 344 kB 62.6 MB/s 
     |████████████████████████████████| 1.6 MB 37.3 MB/s 
     |████████████████████████████████| 51 kB 7.8 MB/s 
  Created wheel for py-vox-io: filename=py_vox_io-0.1-py3-none-any.whl size=5719 sha256=9fcbad5f4d5ac9991b6b6ea5bd77539393963807f64b9cb935189d0354839899
  Stored in directory: /root/.cache/pip/wheels/65/81/

In [110]:
"""
IMPORTANTE
Para el correcto funcionamiento de los import, es necesario eliminar 
la linea: "from quad_mesh_simplify import simplify_mesh" del 
archivo mesh.py.
"""
from voxelfuse.voxel_model import VoxelModel
from voxelfuse.primitives import generateMaterials
from voxelfuse.mesh import Mesh

In [132]:
def VoxToSTL(obj, filename):
  if __name__=='__main__':
    model = VoxelModel(obj, generateMaterials(4))  #4 is aluminium.
    mesh = Mesh.fromVoxelModel(model)
    mesh.export(f'{filename}.stl')

In [134]:
VoxToSTL(x_val[k].reshape(32,32,32), 'broken')

Meshing: 100%|██████████| 2070/2070 [00:00<00:00, 5946.62it/s]


In [135]:
VoxToSTL((prediction[k] - x_val[k]).reshape(32,32,32), 'patch')

Meshing: 100%|██████████| 168/168 [00:00<00:00, 5440.92it/s]
